# 07. User Study (Preparation)

We conduct a user study to evaluate V-XNLI.

There are some strategies for the V-NLI user study.
Srinivasan et al. summarize them in [this paper](https://arjun010.github.io/static/papers/nli-eval-cga20.pdf).
We adopt the most straightforward method: target replication.
It doesn't imitate the actual world usage.
However, it's low risk, low effort and suitable for evaluating low-level system features (e.g. sorting).

We split the sixteen participants into groups A and B (The group size is the same).
At first, we share the link to Google Colab ([group A notebook](https://colab.research.google.com/drive/1LBUIO9zSZvBHDjkmn3NaZJiY_GEM6f3O?usp=share_link
) and [group B notebook](https://colab.research.google.com/drive/1UxoMygaIkMv6JuLJJGjA6iwS5fQZyAqs?usp=share_link)) with the participants.
We write all explanations into the notebook, so they can solve the tasks whenever they want.

The task procedure is as follows.
At first, they answer a pre-experiment questionnaire, which asks the basic information (age, gender, experiences with visualization tools, etc.).
Then, after reading the explanation, the participants in group A solve four exercises and six tasks in order with V-NLI, and the participants in group B do the same with V-XNLI.
We select these tasks from the nvBench dataset (See the "Task Selection" section below in this notebook for the details).
They see a table, a figure and a hint (if needed) for each exercise or task and replicate the figure with V-NLI / V-XNLI.
After solving the tasks, they answer a questionnaire about V-NLI / V-XNLI.
Next, they switched V-NLI and V-XNLI.
The participants in group A solve four exercises and six tasks with V-XNLI, and the participants in group B do the same with V-NLI.
After that, they answer a questionnaire about V-NLI / V-XNLI.
Finally, they take a post-experiment questionnaire.


## Setup

In [1]:
import logging
import sqlite3
import sys

from pathlib import Path
from random import Random
from typing import List, Optional, Tuple

import altair as alt
import numpy as np
import pandas as pd

from IPython.display import display

from vxnli.models.v0.model import Model as BaselineModel
from vxnli.models.v1.model import Model
from vxnli._vega_zero import VegaZero
from vxnli.plot import Plot

plot_logger = logging.getLogger("vxnli.plot")
plot_logger.setLevel(logging.DEBUG)
plot_logger.addHandler(logging.StreamHandler(sys.stdout))


In [2]:
DATA_DIR: Path = Path("../data/")
DATABASE_DIR: Path = DATA_DIR.joinpath("datasets/nvBench/database")


In [3]:
candidates_test_df = pd.read_csv(DATA_DIR.joinpath("datasets/vxnli-v0/test.csv"))
candidates_val_df = pd.read_csv(DATA_DIR.joinpath("datasets/vxnli-v0/val.csv"))

candidates_test_df["subset"] = "test"
candidates_val_df["subset"] = "val"

candidates_df = pd.concat([candidates_test_df, candidates_val_df])

candidates_df = candidates_df[
    ["subset", "db_id", "table", "chart", "hardness", "vega_zero"]
]
candidates_df = candidates_df.drop_duplicates()
candidates_df = candidates_df.sample(frac=1, random_state=123)
candidates_df = candidates_df.reset_index(drop=True)

candidates_df["chart"] = candidates_df["vega_zero"].apply(
    lambda x: VegaZero.parse(x).mark
)

candidates_df


,subset,db_id,table,chart,hardness,vega_zero
0,test,candidate_poll,people,bar,Easy,mark bar encoding x sex y aggregate min weight...
1,test,company_employee,company,bar,Hard,mark bar encoding x headquarters y aggregate c...
2,test,tracking_share_transactions,transactions,line,Medium,mark line encoding x date_of_transaction y agg...
3,test,riding_club,player,bar,Medium,mark bar encoding x gender y aggregate count g...
4,val,school_finance,school,bar,Medium,mark bar encoding x county y aggregate sum enr...
...,...,...,...,...,...,...
766,test,cinema,film,bar,Medium,mark bar encoding x directed_by y aggregate co...
767,test,tracking_share_transactions,lots,bar,Medium,mark bar encoding x lot_details y aggregate co...
768,test,behavior_monitoring,student_addresses,bar,Medium,mark bar encoding x date_address_to y aggregat...
769,test,tracking_share_transactions,transactions,line,Hard,mark line encoding x date_of_transaction y agg...


In [4]:
vnli = Plot(BaselineModel())
vxnli = Plot(Model())


Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [5]:
def load_table(db_id: str, table: str) -> pd.DataFrame:
    with sqlite3.connect(DATABASE_DIR.joinpath(f"{db_id}/{db_id}.sqlite")) as con:
        return pd.read_sql(f"SELECT * FROM {table}", con)


def preprocess_table(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={col: col.lower() for col in df.columns})

    # df = df.astype(str)

    for col_name, col_dtype in zip(df.columns, df.dtypes):
        # HACK: vega_zero is lower-cased
        if pd.api.types.is_string_dtype(col_dtype):
            df[col_name] = df[col_name].str.lower()

    return df


def show_figure(vega_zero: str, df: pd.DataFrame) -> None:
    df = preprocess_table(df)

    vega_lite = VegaZero.parse(vega_zero)
    vega_lite = vega_lite.to_vega_lite(df)
    vega_lite = alt.Chart.from_dict(vega_lite)

    display(vega_lite)


def check_task(db_id: str, table: str, vega_zero: str, hint: str = None) -> None:
    table = load_table(db_id, table)

    display(table.head())

    show_figure(vega_zero, table)

    if hint is not None:
        print(f"hint: {hint}")

    return table


## Select Tasks

We select 6 tasks + 3 exercise tasks from the nvBench datasets preprocessed in the previous notebook.

We use different tables for each task or exercise task.
We sample tasks from the test subset as much as possible, but we also do from the validation subset because the size of the test subset is insufficient.

After trial and error, we decided the task and exercise patterns as follows.

Exercises:

| Hardness | Chart | Y Aggregation | Transform | Hint |
| :---: | :---: | :---: | :---: | :---: |
| Easy | Bar | Count | --- | --- |
| Easy | Line | --- | --- | --- |
| Easy | Pie | Mean | --- | --- |

Tasks:

| Hardness | Chart | Y Aggregation | Transform | Hint |
| :---: | :---: | :---: | :---: | :---: |
| Easy | Bar | Sum | --- | --- |
| Medium | Line | Count | Sort (X Desc) | --- |
| Medium | Bar | Count | Filter | Yes |
| Medium | Pie | --- | Topk | Yes |
| Hard | Bar | Count | Stacked Bar + Sort (Y Asc) | --- |

There are some caveats.

Firstly, we don't adopt time bin operation because it is too advantageous for V-XNLI.
When you use the time bin operation, the axis label in the figure is like "column_name (week)".
Our V-XNLI model can interpret x="column_name (week)" correctly, but V-NLI can't do that.

Secondly, 3 out of 5 tasks use the count aggregation, but we had no choice but to count for these tasks.
Check the cells below for the details.

Finally, we add hints for the filter and top tasks because it's too hard without them.
We tried to find out the filter or took tasks you can solve without a hint. However, we couldn't.
Check the cells below for the details.


### Tasks

#### Bar Chart (Hard)

In [7]:
hard_bar_candidates_df = candidates_df[
    candidates_df["vega_zero"].str.contains(" color ")
    & candidates_df["vega_zero"].str.contains(" sort ")
    & (~candidates_df["vega_zero"].str.contains(" bin "))
    & (candidates_df["subset"] == "test")
    & (candidates_df["chart"] == "bar")
    & (candidates_df["hardness"] == "Hard")
]

hard_bar_candidates_df


,subset,db_id,table,chart,hardness,vega_zero
1,test,company_employee,company,bar,Hard,mark bar encoding x headquarters y aggregate c...
79,test,company_employee,company,bar,Hard,mark bar encoding x headquarters y aggregate c...
627,test,company_employee,company,bar,Hard,mark bar encoding x headquarters y aggregate c...
648,test,company_employee,company,bar,Hard,mark bar encoding x headquarters y aggregate c...


In [8]:
# Use y asc
hard_bar_task = candidates_df.iloc[627][["db_id", "table", "vega_zero"]].to_dict()
hard_bar_task["hint"] = None
hard_bar_task


{'db_id': 'company_employee',
 'table': 'company',
 'vega_zero': 'mark bar encoding x headquarters y aggregate count headquarters color industry transform group x sort y asc',
 'hint': None}

In [9]:
hard_bar_task_table = check_task(**hard_bar_task)


,Company_ID,Name,Headquarters,Industry,Sales_in_Billion,Profits_in_Billion,Assets_in_Billion,Market_Value_in_Billion
0,1.0,ExxonMobil,USA,Oil and gas,433.5,41.1,331.1,407.4
1,2.0,JPMorgan Chase,USA,Banking,110.8,19.0,2265.8,170.1
2,3.0,General Electric,USA,Conglomerate,147.3,14.2,717.2,213.7
3,4.0,Royal Dutch Shell,Netherlands,Oil and gas,470.2,30.9,340.5,227.6
4,5.0,Industrial and Commercial Bank of China,China,Banking,82.6,25.1,2039.1,237.4


alt.Chart(...)

In [10]:
# Failed: no sort
# vnli(hard_bar_task_table, "a stacked bar chart which shows the total number of records by headquarters. group bars by industry, and sort values in the ascending order.")

# Success
# vnli(hard_bar_task_table, "a stacked bar chart which shows the total number of records by headquarters. group bars by industry, and sort y-axis in the ascending order.")

# Failed: no color
# vnli(hard_bar_task_table, "count records by headquarters with a colored bar graph. color is industry. sort values by the y-axis (asc order).")
# vnli(hard_bar_task_table, "count records by headquarters with a colored bar graph. categorize values in the industry column. sort values by the y-axis (asc order).")

# Success
vnli(
    hard_bar_task_table,
    "count records by headquarters with a colored bar graph, categorized in industry. sort values by the y-axis (asc order).",
)


vega_zero: mark bar encoding x headquarters y aggregate count headquarters color industry transform group x sort y asc


alt.Chart(...)

In [14]:
# Success
# vxnli(hard_bar_task_table, x="headquarters", y="count of records", bar_color="industry", order="ascending")

# Failed: no color
# vxnli(hard_bar_task_table, "stacked bar chart", x_axis="headquarters", y_axis="count", z_axis="industry", y_sort="low to high")

# Failed: no color & no sort
# vxnli(hard_bar_task_table, "colored bar chart", x_axis="headquarters", y_axis="count", z_axis="industry", sorting="up")

# Success



vega_zero: mark bar encoding x headquarters y aggregate count headquarters color industry transform group x sort y asc


alt.Chart(...)

#### Pie Chart

In [15]:
medium_pie_candidates_df = candidates_df[
    candidates_df["vega_zero"].str.contains(" topk ")
    & (candidates_df["chart"] == "arc")
]

medium_pie_candidates_df


,subset,db_id,table,chart,hardness,vega_zero
174,val,college_1,student,arc,Medium,mark arc encoding x stu_fname y aggregate none...


In [16]:
medium_pie_task = candidates_df.iloc[174][["db_id", "table", "vega_zero"]].to_dict()
medium_pie_task["hint"] = "Top 5 STU_GPA"
medium_pie_task


{'db_id': 'college_1',
 'table': 'student',
 'vega_zero': 'mark arc encoding x stu_fname y aggregate none stu_gpa transform sort y desc topk 5',
 'hint': 'Top 5 STU_GPA'}

In [17]:
medium_pie_task_table = check_task(**medium_pie_task)


,STU_NUM,STU_LNAME,STU_FNAME,STU_INIT,STU_DOB,STU_HRS,STU_CLASS,STU_GPA,STU_TRANSFER,DEPT_CODE,STU_PHONE,PROF_NUM
0,321452,Bowser,William,C,1975-2-12,42,So,2.84,0,BIOL,2134,205
1,324257,Smithson,Anne,K,1981-11-15,81,Jr,3.27,1,CIS,2256,222
2,324258,Brewer,Juliette,,1969-8-23,36,So,2.26,1,ACCT,2256,228
3,324269,Oblonski,Walter,H,1976-9-16,66,Jr,3.09,0,CIS,2114,222
4,324273,Smith,John,D,1958-12-30,102,Sr,2.11,1,ENGL,2231,199


alt.Chart(...)

hint: Top 5 STU_GPA


In [18]:
# Success
# vnli(medium_pie_task_table, "show top 5 stu_gpa for each stu_fname with a pie chart.")

# Failed: mark arc encoding x stu_gpa y aggregate count stu_gpa transform group x topk 5
# vnli(medium_pie_task_table, "Give me the top 5 stu_gpa.")

# Failed: mark arc encoding x stu_gpa y aggregate count stu_gpa transform group x
# vnli(medium_pie_task_table, "Give me the top 5 stu_gpa per the student name")

# Failed: mark arc encoding x stu_fname y aggregate count stu_gpa transform group x sort x asc topk 5
# vnli(medium_pie_task_table, "Show me the proportion of stu_gpa by stu_fname in a circle chart. limit top 5 results.")

# Success
# vnli(medium_pie_task_table, "show stu_gpa by stu_fname with a pie chart. limit top 5 results")

# Success
vnli(medium_pie_task_table, "draw a pie chart. stu_fname and stu_gpa. top 5 results.")


vega_zero: mark arc encoding x stu_fname y aggregate none stu_gpa transform sort y desc topk 5


alt.Chart(...)

In [21]:
# V-XNLI

# Success
vxnli(medium_pie_task_table, proportion="stu_gpa", axis="stu_fname", limit="best 5")

# Success
# vxnli(medium_pie_task_table, proportion="stu_gpa", axis="stu_fname", best=5)

# Success
# vxnli(medium_pie_task_table, chart="circle", y="stu_gpa", x="stu_fname", top=5)


vega_zero: mark arc encoding x stu_fname y aggregate none stu_gpa transform sort y desc topk 5


alt.Chart(...)

#### Bar Chart (Medium)

In [14]:
# 571 was the first candidate because you can solve this task without hint
# However, I failed to predict the result well with V-NLI even though I can do it easily with V-XNLI
# Besides, I noticed topk operation can be used

# Failed: scatter chart displayed (mark point encoding x name y aggregate none number_deaths transform sort y desc)
# vnli(medium_bar_task_table, "tell me the relationship between name and number_deaths in a bar chart. number_deaths >= 1.")

# Almost Success: mark bar encoding x name y aggregate none number_deaths transform filter number_deaths > 1
# vnli(medium_bar_task_table, "Show the number_deaths for each name in a bar chart. number_deaths must be bigger or equal than 1.")

# Success
# vnli(medium_bar_task_table, "Show the number_deaths for each name in a bar chart. number_deaths must be greater or equal than 1.")

# Failed: mark bar encoding x name y aggregate none number_deaths transform > 1
# vnli(medium_bar_task_table, "Show the number_deaths for each name in a bar chart. number_deaths >= 1.")

# Failed: mark bar encoding x name y aggregate none number_deaths transform sort max_speed asc
# vnli(medium_bar_task_table, "Show the number_deaths for each name in a bar chart. the value must be at least 1.")

# Failed: mark bar encoding x name y aggregate none number_deaths transform sort x asc
# vnli(medium_bar_task_table, "Show the number_deaths (>= 1) for each name in a bar chart")

# Failed: mark bar encoding x name y aggregate none number_deaths transform filter damage_millions_usd!= "null"
# vnli(medium_bar_task_table, "Show the number_deaths over name with a histogram. The number_deaths is not 0.")


In [22]:
medium_bar_candidates_df = candidates_df[
    candidates_df["vega_zero"].str.contains(" filter ")
    & (candidates_df["vega_zero"].str.contains(" = "))
    & (~candidates_df["vega_zero"].str.contains(" sort "))
    & (~candidates_df["vega_zero"].str.contains(" bin "))
    & (candidates_df["chart"] == "bar")
    & (candidates_df["hardness"] == "Medium")
    # Test subset is not enough
    # & (candidates_df["subset"] == "test")
]

medium_bar_candidates_df


,subset,db_id,table,chart,hardness,vega_zero
361,test,behavior_monitoring,behavior_incident,bar,Medium,mark bar encoding x date_incident_end y aggreg...
464,val,activity_1,faculty,bar,Medium,mark bar encoding x sex y aggregate count sex ...
664,val,college_1,professor,bar,Medium,mark bar encoding x dept_code y aggregate coun...


In [23]:
# 361 is a flat bar chart (value == 1 count of recrods)
medium_bar_task = candidates_df.iloc[464][["db_id", "table", "vega_zero"]].to_dict()

# If Rank = 'AsstProf', you can copy & paste this with V-NLI
# So we use a complex text instead
medium_bar_task[
    "hint"
] = "Check the Rank column. This figure uses the rows in which the value is 'AsstProf'."
medium_bar_task


{'db_id': 'activity_1',
 'table': 'faculty',
 'vega_zero': 'mark bar encoding x sex y aggregate count sex transform filter rank = "asstprof" group x',
 'hint': "Check the Rank column. This figure uses the rows in which the value is 'AsstProf'."}

In [24]:
medium_bar_task_table = check_task(**medium_bar_task)


,FacID,Lname,Fname,Rank,Sex,Phone,Room,Building
0,1082,Giuliano,Mark,Instructor,M,2424,224,NEB
1,1121,Goodrich,Michael,Professor,M,3593,219,NEB
2,1148,Masson,Gerald,Professor,M,3402,224B,NEB
3,1172,Runolfsson,Thordur,AssocProf,M,3121,119,Barton
4,1177,Naiman,Daniel,Professor,M,3571,288,Krieger


alt.Chart(...)

hint: Check the Rank column. This figure uses the rows in which the value is 'AsstProf'.


In [28]:
# Success
vnli(medium_bar_task_table, "a bar graph which shows the number of records per gender. however, the rank must be 'AsstProf'")

# Failed (mark bar encoding x asstprof y aggregate count asstprof transform filter rank = "asstprof" group x)
# vnli(medium_bar_task_table, "a bar graph which shows how many people, who has the rank 'AsstProf', by gender")

# Success
# vnli(medium_bar_task_table, "a bar graph which shows how many people, who has the rank 'AsstProf', for each sex")

# Success
# vnli(
#     medium_bar_task_table,
#     "show me the total number of records by sex. filter results with rank = 'AsstProf'",
# )


vega_zero: mark bar encoding x sex y aggregate count sex transform filter rank = "asstprof" group x


alt.Chart(...)

In [25]:
# Failed: mark bar encoding x sex y aggregate count sex transform group x
# vxnli(medium_bar_task_table, "count records by each gender", rank="AsstProf")

# Success
# vxnli(medium_bar_task_table, "count records by each gender", filter="rank = AsstProf")

# Success
# vxnli(medium_bar_task_table, count_by="sex", condition="rank is 'AsstProf'")

# Success
vxnli(medium_bar_task_table, x="sex", y="count of records", where="rank == 'AsstProf'")


vega_zero: mark bar encoding x sex y aggregate count sex transform filter rank = "asstprof" group x


alt.Chart(...)

#### Line Chart


In [29]:
medium_line_candidates_df = candidates_df[
    # candidates_df["vega_zero"].str.contains(" sort x ")
    (~candidates_df["vega_zero"].str.contains(" filter "))
    & (~candidates_df["vega_zero"].str.contains(" bin "))
    & (~candidates_df["vega_zero"].str.contains(" color "))
    & (candidates_df["chart"] == "line")
    # & (candidates_df["subset"] == "test")
    & (candidates_df["hardness"] == "Medium")
]

medium_line_candidates_df


,subset,db_id,table,chart,hardness,vega_zero
59,val,game_injury,game,line,Medium,mark line encoding x season y aggregate count ...
76,test,behavior_monitoring,student_addresses,line,Medium,mark line encoding x date_address_to y aggrega...
137,val,cre_Doc_Tracking_DB,document_locations,line,Medium,mark line encoding x date_in_location_from y a...
160,test,behavior_monitoring,student_addresses,line,Medium,mark line encoding x date_address_to y aggrega...
197,val,game_injury,game,line,Medium,mark line encoding x season y aggregate count ...
368,test,behavior_monitoring,student_addresses,line,Medium,mark line encoding x date_address_to y aggrega...
429,val,cre_Doc_Tracking_DB,document_locations,line,Medium,mark line encoding x date_in_locaton_to y aggr...
484,val,cre_Doc_Tracking_DB,document_locations,line,Medium,mark line encoding x date_in_locaton_to y aggr...
621,val,cre_Doc_Tracking_DB,document_locations,line,Medium,mark line encoding x date_in_location_from y a...


In [30]:
# 59 sort asc = 197 desc,
# 76 is not bad, but the column is date (not year, as string, complicated)
# 137 is the same

# 197
medium_line_task = candidates_df.iloc[197][["db_id", "table", "vega_zero"]].to_dict()
medium_line_task["hint"] = None
medium_line_task


{'db_id': 'game_injury',
 'table': 'game',
 'vega_zero': 'mark line encoding x season y aggregate count season transform group x sort x desc',
 'hint': None}

In [31]:
medium_line_task_table = check_task(**medium_line_task)


,stadium_id,id,Season,Date,Home_team,Away_team,Score,Competition
0,1,1,2007,18 May 2007,Quruvchi,Pakhtakor,1–1,League
1,2,2,2007,22 September 2007,Pakhtakor,Quruvchi,0–0,League
2,3,3,2007,9 December 2007,Pakhtakor,Quruvchi,0–0 (7:6),Cup
3,4,4,2008,10 July 2008,Pakhtakor,Quruvchi,1–1,League
4,5,5,2008,16 August 2008,Bunyodkor,Pakhtakor,1–1,League


alt.Chart(...)

In [32]:
# Success
# vnli(medium_line_task_table, "line chart. season and the count of records. sort x from high to low")

# Success
# vnli(medium_line_task_table, "line chart. season and the count of records. descending order")

# Success
vnli(
    medium_line_task_table,
    "draw a line graph which shows the number of records by the season, and the order is desc",
)


vega_zero: mark line encoding x season y aggregate count season transform group x sort x desc


alt.Chart(...)

In [35]:
# Success
vxnli(medium_line_task_table, chart="line", x="season", y="count of records", sort_axis="x", sort_order="desc")

# Failed: mark line encoding x season y aggregate count season transform group x sort y desc
# vxnli(medium_line_task_table, "plot a line chart", "sort season from high to low", count_records_by="season")

# Success
# vxnli(
#     medium_line_task_table,
#     "plot a line chart",
#     "sort x from high to low",
#     count_records_by="season",
# )


vega_zero: mark line encoding x season y aggregate count season transform group x sort x desc


alt.Chart(...)

#### Bar Chart (Easy)

In [166]:
easy_bar_candidates_df = candidates_df[
    (~candidates_df["vega_zero"].str.contains(" sort "))
    & (~candidates_df["vega_zero"].str.contains(" filter "))
    & (~candidates_df["vega_zero"].str.contains(" bin "))
    & (~candidates_df["vega_zero"].str.contains(" aggregate none "))
    & (~candidates_df["vega_zero"].str.contains(" aggregate count "))
    & (candidates_df["chart"] == "bar")
    & (candidates_df["subset"] == "test")
]

easy_bar_candidates_df


,subset,db_id,table,chart,hardness,vega_zero
183,test,candidate_poll,people,bar,Easy,mark bar encoding x sex y aggregate sum height...
220,test,candidate_poll,people,bar,Easy,mark bar encoding x sex y aggregate min weight...
248,test,train_station,station,bar,Easy,mark bar encoding x location y aggregate sum n...
255,test,candidate_poll,people,bar,Easy,mark bar encoding x sex y aggregate mean weigh...
281,test,candidate_poll,people,bar,Easy,mark bar encoding x sex y aggregate sum weight...
285,test,candidate_poll,people,bar,Easy,mark bar encoding x sex y aggregate mean heigh...
353,test,cinema,cinema,bar,Easy,mark bar encoding x openning_year y aggregate ...
377,test,cinema,cinema,bar,Easy,mark bar encoding x openning_year y aggregate ...
407,test,customers_and_invoices,financial_transactions,bar,Easy,mark bar encoding x transaction_type y aggrega...
427,test,tracking_share_transactions,transactions,bar,Easy,mark bar encoding x transaction_type_code y ag...


In [107]:
# 183, sum of height is a bit strange, 220 is acceptable, but sex is already used
easy_bar_task = candidates_df.iloc[248][["db_id", "table", "vega_zero"]].to_dict()
easy_bar_task["hint"] = None
easy_bar_task


{'db_id': 'train_station',
 'table': 'station',
 'vega_zero': 'mark bar encoding x location y aggregate sum number_of_platforms transform group x',
 'hint': None}

In [108]:
easy_bar_task_table = check_task(**easy_bar_task)


,Station_ID,Name,Annual_entry_exit,Annual_interchanges,Total_Passengers,Location,Main_Services,Number_of_Platforms
0,1,London Waterloo,94.046,9.489,103.534,London,South Western Main Line West of England Main Line,19
1,2,London Victoria,76.231,9.157,85.380,London,Brighton Main Line Chatham Main Line,19
2,3,London Bridge,52.634,8.742,61.376,London,South Eastern Main Line Thameslink,12
3,4,London Liverpool Street,57.107,2.353,59.460,London,Great Eastern Main Line West Anglia Main Line,18
4,5,London Euston,36.609,3.832,40.440,London,West Coast Main Line,18


alt.Chart(...)

In [115]:
# Success
# vnli(easy_bar_task_table, "location and sum of number_of_platforms in a bar chart")

# Failed: mark bar encoding x location y aggregate count location transform group x
# vnli(easy_bar_task_table, "how many platforms in each location?")

# Success
# vnli(easy_bar_task_table, "the sum number_of_platforms in each location?")

vnli(easy_bar_task_table, "sum of number_of_platform vs location")


vega_zero: mark bar encoding x location y aggregate sum number_of_platforms transform group x


alt.Chart(...)

In [116]:
# Success
# vxnli(easy_bar_task_table, "sum of number_of_platform vs location")

# Success
# vxnli(easy_bar_task_table, ["location", "sum(number_of_platform)"])


vega_zero: mark bar encoding x location y aggregate sum number_of_platforms transform group x


alt.Chart(...)

### Exercises


#### Pie Chart

In [268]:
pie_exercise_candidates_df = candidates_df[
    (~candidates_df["vega_zero"].str.contains(" filter "))
    & (~candidates_df["vega_zero"].str.contains(" topk "))
    # agg none is not available in the first place
    & (~candidates_df["vega_zero"].str.contains(" count "))  # Hint can be a bit strange
    & (~candidates_df["vega_zero"].str.contains(" sum "))  # Used in the next
    & (candidates_df["chart"] == "arc")
    & (candidates_df["subset"] == "test")
]

pie_exercise_candidates_df


,subset,db_id,table,chart,hardness,vega_zero
460,test,candidate_poll,people,arc,Easy,mark arc encoding x sex y aggregate mean weigh...
574,test,tracking_share_transactions,transactions,arc,Easy,mark arc encoding x transaction_type_code y ag...
743,test,candidate_poll,people,arc,Easy,mark arc encoding x sex y aggregate min weight...


In [269]:
# 460: gender is already used
pie_exercise = candidates_df.iloc[574][["db_id", "table", "vega_zero"]].to_dict()
pie_exercise["hint"] = "Mean of amount_of_transaction"
pie_exercise


{'db_id': 'tracking_share_transactions',
 'table': 'transactions',
 'vega_zero': 'mark arc encoding x transaction_type_code y aggregate mean amount_of_transaction transform group x',
 'hint': 'Mean of amount_of_transaction'}

In [139]:
pie_exercise_table = check_task(**pie_exercise)


,transaction_id,investor_id,transaction_type_code,date_of_transaction,amount_of_transaction,share_count,other_details
0,1,6,SALE,1988-09-16 19:02:51,302507.6996,8718572,None
1,2,18,PUR,1982-06-06 17:19:00,27.2570,9,None
2,3,2,SALE,1979-04-27 06:03:59,48777.9690,8580,None
3,4,14,PUR,2001-11-28 15:06:25,4.5263,8040,None
4,5,8,PUR,1977-08-17 13:13:30,0.0000,930,None


alt.Chart(...)

hint: Mean of amount_of_transaction


In [145]:
# Success
# vnli(pie_exercise_table, "show me the proportion of the average amount of transaction, and group by transaction type code")

# Failed: mark point encoding x amount_of_transaction y aggregate mean amount_of_transaction transform group x
# vnli(pie_exercise_table, "transaction_type_code and mean of amount_of_transaction with a circle graph")

# Success
vnli(
    pie_exercise_table,
    "transaction_type_code and mean of amount_of_transaction in a pie chart",
)


vega_zero: mark arc encoding x transaction_type_code y aggregate mean amount_of_transaction transform group x


alt.Chart(...)

In [147]:
# Success
# vxnli(pie_exercise_table, x="transaction_type_code", y=" mean of amount_of_transaction", plot="pie")

# Success
vxnli(
    pie_exercise_table,
    "transaction_type_code and mean of amount_of_transaction",
    graph="circle",
)


vega_zero: mark arc encoding x transaction_type_code y aggregate mean amount_of_transaction transform group x


alt.Chart(...)

#### Line Chart

In [198]:
line_exercise_candidates_df = candidates_df[
    (~candidates_df["vega_zero"].str.contains(" bin "))
    & (~candidates_df["vega_zero"].str.contains(" sort "))
    & (candidates_df["chart"] == "line")
    & (candidates_df["subset"] == "test")
    # & (candidates_df["hardness"] == "Easy")
]

line_exercise_candidates_df


,subset,db_id,table,chart,hardness,vega_zero


In [271]:
line_exercise_candidates_df = pd.read_csv(
    DATA_DIR.joinpath("datasets/vxnli-v0/train.csv")
)

line_exercise_candidates_df = line_exercise_candidates_df[
    ["chart", "hardness", "db_id", "table", "vega_zero"]
]

line_exercise_candidates_df = line_exercise_candidates_df.drop_duplicates()
line_exercise_candidates_df = line_exercise_candidates_df[
    line_exercise_candidates_df["chart"] == "Line"
]

line_exercise_candidates_df = line_exercise_candidates_df[
    (line_exercise_candidates_df["hardness"] == "Easy")
    & (~line_exercise_candidates_df["vega_zero"].str.contains(" bin "))
    & (~line_exercise_candidates_df["vega_zero"].str.contains(" sort "))
    & (~line_exercise_candidates_df["vega_zero"].str.contains(" filter "))
    & (
        line_exercise_candidates_df["vega_zero"].str.contains(" count ")
        | line_exercise_candidates_df["vega_zero"].str.contains(" none ")
    )
]

line_exercise_candidates_df = line_exercise_candidates_df.reset_index(drop=True)

line_exercise_candidates_df


,chart,hardness,db_id,table,vega_zero
0,Line,Easy,dog_kennels,dogs,mark line encoding x date_arrived y aggregate ...
1,Line,Easy,employee_hire_evaluation,hiring,mark line encoding x start_from y aggregate no...
2,Line,Easy,employee_hire_evaluation,hiring,mark line encoding x start_from y aggregate no...
3,Line,Easy,hr_1,employees,mark line encoding x hire_date y aggregate non...
4,Line,Easy,apartment_rentals,apartment_bookings,mark line encoding x booking_end_date y aggreg...
5,Line,Easy,baseball_1,hall_of_fame,mark line encoding x yearid y aggregate count ...
6,Line,Easy,bike_1,station,mark line encoding x installation_date y aggre...
7,Line,Easy,insurance_policies,settlements,mark line encoding x date_claim_settled y aggr...
8,Line,Easy,e_learning,student_course_enrolment,mark line encoding x date_of_completion y aggr...
9,Line,Easy,department_management,department,mark line encoding x creation y aggregate coun...


In [272]:
# 0 flat
# 1 employee_id is strange..., but the others are also not good. accept it
line_exercise = line_exercise_candidates_df.iloc[1][
    ["db_id", "table", "vega_zero"]
].to_dict()
line_exercise["hint"] = None
line_exercise


{'db_id': 'employee_hire_evaluation',
 'table': 'hiring',
 'vega_zero': 'mark line encoding x start_from y aggregate none employee_id',
 'hint': None}

In [273]:
line_exercise_table = check_task(**line_exercise)


,Shop_ID,Employee_ID,Start_from,Is_full_time
0,1,1,2009,T
1,1,2,2003,T
2,8,3,2011,F
3,4,4,2012,T
4,5,5,2013,T


alt.Chart(...)

In [267]:
# Success
vnli(line_exercise_table, "start_from and employee_id in a line chart")


vega_zero: mark line encoding x start_from y aggregate none employee_id


alt.Chart(...)

In [266]:
# Success
vxnli(line_exercise_table, {"x": "start from", "y": "employee_id"})


vega_zero: mark line encoding x start_from y aggregate none employee_id


alt.Chart(...)

#### Bar Chart

In [276]:
bar_exercise_candidates_df = candidates_df[
    (~candidates_df["vega_zero"].str.contains(" filter "))
    & (~candidates_df["vega_zero"].str.contains(" sort "))
    # agg none is not available in the first place
    & (candidates_df["vega_zero"].str.contains(" count "))  # Hint can be a bit strange
    & (candidates_df["chart"] == "bar")
    & (candidates_df["hardness"] == "Easy")
    & (candidates_df["subset"] == "test")
]

bar_exercise_candidates_df


,subset,db_id,table,chart,hardness,vega_zero
149,test,riding_club,player,bar,Easy,mark bar encoding x gender y aggregate count g...
209,test,riding_club,player,bar,Easy,mark bar encoding x occupation y aggregate cou...
215,test,wta_1,matches,bar,Easy,mark bar encoding x year y aggregate count yea...
223,test,tracking_share_transactions,lots,bar,Easy,mark bar encoding x lot_details y aggregate co...
230,test,train_station,train,bar,Easy,mark bar encoding x name y aggregate count nam...
237,test,behavior_monitoring,assessment_notes,bar,Easy,mark bar encoding x date_of_notes y aggregate ...
261,test,train_station,station,bar,Easy,mark bar encoding x location y aggregate count...
269,test,company_employee,company,bar,Easy,mark bar encoding x headquarters y aggregate c...
283,test,local_govt_in_alabama,participants,bar,Easy,mark bar encoding x participant_type_code y ag...
320,test,tracking_share_transactions,investors,bar,Easy,mark bar encoding x investor_details y aggrega...


In [284]:
bar_exercise = candidates_df.iloc[209][["db_id", "table", "vega_zero"]].to_dict()
bar_exercise["hint"] = None
bar_exercise


{'db_id': 'riding_club',
 'table': 'player',
 'vega_zero': 'mark bar encoding x occupation y aggregate count occupation transform group x',
 'hint': None}

In [285]:
bar_exercise_table = check_task(**bar_exercise)


,Player_ID,Sponsor_name,Player_name,Gender,Residence,Occupation,Votes,Rank
0,1,Brandon—Souris,Jean Luc Bouché,M,Brandon,Locomotive Engineer,6055,2nd
1,2,Charleswood—St. James—Assiniboia,Fiona Shiells,F,Winnipeg,Ministerial Assistant,7190,3rd
2,3,Churchill,Niki Ashton,F,Thompson,Researcher,8734,1st
3,4,Dauphin—Swan River—Marquette,Ron Strynadka,M,Birtle,Retired,4914,2nd
4,5,Elmwood—Transcona,Jim Maloway,M,Winnipeg,Small Businessman,14355,1st


alt.Chart(...)

In [286]:
# Success
vnli(bar_exercise_table, "count records by occupation")


vega_zero: mark bar encoding x occupation y aggregate count occupation transform group x


alt.Chart(...)

In [287]:
# Success
vxnli(bar_exercise_table, "count records by occupation")


vega_zero: mark bar encoding x occupation y aggregate count occupation transform group x


alt.Chart(...)

### Summary

In [117]:
TASKS = [
    easy_bar_task,
    medium_line_task,
    medium_bar_task,
    medium_pie_task,
    hard_bar_task,
]

TASKS


[{'db_id': 'train_station',
  'table': 'station',
  'vega_zero': 'mark bar encoding x location y aggregate sum number_of_platforms transform group x',
  'hint': None},
 {'db_id': 'game_injury',
  'table': 'game',
  'vega_zero': 'mark line encoding x season y aggregate count season transform group x sort x desc',
  'hint': None},
 {'db_id': 'activity_1',
  'table': 'faculty',
  'vega_zero': 'mark bar encoding x sex y aggregate count sex transform filter rank = "asstprof" group x',
  'hint': "Check the Rank column. This figure uses the rows in which the value is 'AsstProf'."},
 {'db_id': 'college_1',
  'table': 'student',
  'vega_zero': 'mark arc encoding x stu_fname y aggregate none stu_gpa transform sort y desc topk 5',
  'hint': 'Top 5 STU_GPA'},
 {'db_id': 'company_employee',
  'table': 'company',
  'vega_zero': 'mark bar encoding x headquarters y aggregate count headquarters color industry transform group x sort y asc',
  'hint': None}]

In [288]:
EXERCISES = [
    bar_exercise,
    line_exercise,
    pie_exercise,
]

EXERCISES


[{'db_id': 'riding_club',
  'table': 'player',
  'vega_zero': 'mark bar encoding x occupation y aggregate count occupation transform group x',
  'hint': None},
 {'db_id': 'employee_hire_evaluation',
  'table': 'hiring',
  'vega_zero': 'mark line encoding x start_from y aggregate none employee_id',
  'hint': None},
 {'db_id': 'tracking_share_transactions',
  'table': 'transactions',
  'vega_zero': 'mark arc encoding x transaction_type_code y aggregate mean amount_of_transaction transform group x',
  'hint': 'Mean of amount_of_transaction'}]

In [ ]:
# Check if all db_ids are different
assert len({t["db_id"] for t in (TASKS + EXERCISES)}) == len(TASKS + EXERCISES)
